In [1]:
import os

from IPython import display
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import csv
from functools import reduce

import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import tensorflow_model_optimization as tfmot

import params as yamnet_params
import yamnet as yamnet_model

In [2]:
# load in the yamnet model
params = yamnet_params.Params()
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights('yamnet.h5')
yamnet_classes = yamnet_model.class_names('yamnet_class_map.csv')

In [12]:
yamnet.summary()

Model: "yamnet_frames"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
waveform_binary 0 (InputLayer)  [(15600,)]           0                                            
__________________________________________________________________________________________________
tf.signal.frame (TFOpLambda)    (96, 400)            0           waveform_binary 0[0][0]          
__________________________________________________________________________________________________
tf.math.multiply (TFOpLambda)   (96, 400)            0           tf.signal.frame[0][0]            
__________________________________________________________________________________________________
tf.compat.v1.shape (TFOpLambda) (2,)                 0           tf.math.multiply[0][0]           
______________________________________________________________________________________

In [8]:
for i, layer in enumerate(yamnet.layers):
    print(i, layer.name)

0 waveform_binary 0
1 tf.signal.frame
2 tf.math.multiply
3 tf.compat.v1.shape
4 tf.__operators__.getitem
5 tf.math.subtract
6 tf.compat.v1.floor_div
7 tf.math.subtract_1
8 tf.math.subtract_2
9 tf.compat.v1.pad
10 tf.linalg.matmul
11 tf.linalg.matmul_1
12 tf.math.multiply_1
13 tf.math.multiply_2
14 tf.math.add
15 tf.math.sqrt
16 tf.linalg.matmul_2
17 tf.__operators__.add
18 tf.math.log
19 tf.signal.frame_1
20 reshape
21 layer1/conv
22 layer1/conv/bn
23 tf.nn.relu6
24 layer2/depthwise_conv
25 layer2/depthwise_conv/bn
26 tf.nn.relu6_1
27 layer2/pointwise_conv
28 layer2/pointwise_conv/bn
29 tf.nn.relu6_2
30 layer3/depthwise_conv
31 layer3/depthwise_conv/bn
32 tf.nn.relu6_3
33 layer3/pointwise_conv
34 layer3/pointwise_conv/bn
35 tf.nn.relu6_4
36 layer4/depthwise_conv
37 layer4/depthwise_conv/bn
38 tf.nn.relu6_5
39 layer4/pointwise_conv
40 layer4/pointwise_conv/bn
41 tf.nn.relu6_6
42 layer5/depthwise_conv
43 layer5/depthwise_conv/bn
44 tf.nn.relu6_7
45 layer5/pointwise_conv
46 layer5/pointwi

In [9]:
names = set()
for i, layer in enumerate(yamnet.layers):
    if i >=21:
        print(layer)
        names.add(layer.name)

In [10]:
def apply_quant_to_layers(layer):
    if layer.name in names:
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

In [11]:
yamnet_annotated = tf.keras.models.clone_model(
    yamnet,
    clone_function=apply_quant_to_layers
)
yamnet_quant = tfmot.quantization.keras.quantize_apply(yamnet_annotated)


ValueError: Shape must be rank 4 but is rank 5 for '{{node layer2/depthwise_conv/depthwise}} = DepthwiseConv2dNative[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1]](Placeholder, layer2/depthwise_conv/depthwise/ReadVariableOp)' with input shapes: [1,1,48,32,32], [3,3,32,1].

In [19]:
# convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(yamnet_quant)
# converter = tf.lite.TFLiteConverter.from_keras_model(yamnet)
tflite_model = converter.convert()
# save to file called me/yamnet.tflite
# with open('models/3/me/features.tflite', 'wb') as f:
with open('/models/3/me/yamnet_quant.tflite', 'wb') as f:
    f.write(tflite_model)

FileNotFoundError: [Errno 2] No such file or directory: 'models/3/me/yamnet_quant.tflite'

In [22]:
with open('models/3/yamnet_quant.tflite', 'wb') as f:
    f.write(tflite_model)

In [23]:
yamnet_quant.summary()

Model: "yamnet_frames"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
waveform_binary 0 (InputLayer)  [(15600,)]           0                                            
__________________________________________________________________________________________________
tf.signal.frame (TFOpLambda)    (96, 400)            0           waveform_binary 0[0][0]          
__________________________________________________________________________________________________
tf.math.multiply (TFOpLambda)   (96, 400)            0           tf.signal.frame[1][0]            
__________________________________________________________________________________________________
tf.compat.v1.shape (TFOpLambda) (2,)                 0           tf.math.multiply[1][0]           
______________________________________________________________________________________